# Data compilation from database scan

The trained models have been used for scaning the whole database looking for incentives.\
Each sentence has been firs classified into one of to categories, *Incentive* or *Not-Incentive*.\
Sentences which were classified as incentives were further classified into one of the six policy instruments.

Thus 
* Number of sentences per document
* Average length of sentences
* Length of smallest sentence
* Length of longest sentence
* Relative distance of the first incentive from the head of the document
* Distance to the closest previous incentive


In [1]:
import boto3
import csv
import json

In [2]:
# path = 'C:/Users/jordi/Documents/claus/'
# path = '/home/propietari/Documents/claus/'
path = 'C:/Users/hrpeg/Documents/claus/'
filename = 'AWS_S3_keys_wri.json'
file = path + filename
with open(file, 'r') as dict:
    credentials = json.load(dict)
                                      
KEY = list(credentials)[0]
SECRET = list(credentials.values())[0]
# s3BucketName = "wri-testing"
s3BucketName = "wri-nlp-policy"
# region = 'eu-central-1'
region = "us-east-1"

s3 = boto3.resource(
    service_name = 's3',
    region_name = region,
    aws_access_key_id = KEY,
    aws_secret_access_key = SECRET
)

In [3]:
def sentence_stats(sentence_text, min_sentence_length, max_sentence_length, sum_sentence_length):
    sum_sentence_length = sum_sentence_length + len(sentence_text)
    if len(sentence_text) < min_sentence_length:
        min_sentence_length = len(sentence_text)
    if len(sentence_text) > max_sentence_length:
        max_sentence_length = len(sentence_text)
    return min_sentence_length, max_sentence_length, sum_sentence_length


# def sentences_processing()    

In [4]:
# path = "/home/propietari/Documents/GitHub/policy-data-analyzer/tasks/database_analysis/input/"
path = "C:/Users/hrpeg/Documents/GitHub/policy-data-analyzer/tasks/database_analysis/input/"
file_name = "ElSalvadorKeys.json"
file_name = "english_new_document_sentences.json"
file = path + file_name

with open(file, 'r') as fp:
    filter_keys = json.load(fp)

In [5]:
language = "english"
model = ""#"simple_transformers/"
in_prefix = f"{language}_documents/HSSC/{model}sentences/"
out_prefix = f"{language}_documents/HSSC/{model}updated_sentences/"

i = 0
j = 0
documents_with_incentives = 0
results_list = [["file_name", "n_incentives", "Supplies", "Loan", "Fine", "Direct payment", "Technical assistance", "Tax benefit"]]
incentives_dict = {}

for obj in s3.Bucket(s3BucketName).objects.all().filter(Prefix = in_prefix):#.:
    if ".json" in obj.key and "ing" not in obj.key:
        i += 1
        name = obj.key.split("/")[-1].split("_")[0]
        if i % 1000 == 0:
            print(i,  end='\r')
        if name in filter_keys:
            j += 1
            print(j, end = "\r")
#             print(name)
            
#           output.clear()
#         if i < 100000:
    #         print(f"\n\nFile number: {i}\n\n")
#             print(i,  end='\r')
#             name = obj.key.split("/")[-1].split("_")[0]
            flag = False
            sentences = json.loads(obj.get()['Body'].read().decode('utf-8'))
            incentive_counter = 0
            sentences_counter = 0
            min_sentence_length = 100000
            max_sentence_length = 0
            sum_sentence_length = 0
            last_sentence_position = 0
            incentives_dict = {'Supplies': 0,
                               'Loan': 0,
                               'Fine': 0,
                               'Direct payment': 0,
                               'Technical assistance': 0,
                               'Tax benefit': 0}
            for sentence_id in sentences[name]["sentences"]:
                sentences_counter += 1
                sentence = sentences[name]["sentences"][sentence_id]["text"]
                min_sentence_length, max_sentence_length, sum_sentence_length = sentence_stats(sentence, min_sentence_length, max_sentence_length, sum_sentence_length)
                incentive = sentences[name]["sentences"][sentence_id]["label"]  
                if len(incentive) > 1 and incentive[0] == "Incentive":
                    incentive_counter += 1
                    sentence_number = sentence_id.split("_")[2]
                    policy = sentences[name]["sentences"][sentence_id]["label"][1]
                    if last_sentence_position == 0:
                        results_list.append([])
                        last_sentence_position = sentence_number
                        sentences[name]["sentences"][sentence_id]["closest_sentence"] = 0
                        flag = True
                    else:
                        sentences[name]["sentences"][sentence_id]["closest_sentence"] = int(sentence_number) - int(last_sentence_position)
                        last_sentence_position = sentence_number

                    relative_position = int(sentence_number) / sentences[name]["metadata"]["n_sentences"]
                    sentences[name]["sentences"][sentence_id]["relative_position"] = round(relative_position, 2)
                    incentives_dict[policy] += 1
    #                 print(incentive, "\n", sentence_id, " ** ", last_sentence_position)

            # Updating the dictionary
            sentences[name]["metadata"]["min_sent_length"] = min_sentence_length
            sentences[name]["metadata"]["max_sent_length"] = max_sentence_length
            sentences[name]["metadata"]["avg_sent_length"] = int(sum_sentence_length/sentences_counter)
            sentences[name]["metadata"]["number_of_incentives"] = incentive_counter
            for key, value in incentives_dict.items():
                sentences[name]["metadata"][key] = value
            s3.Object(s3BucketName, f"{out_prefix}{name}_sents.json").put(Body = (json.dumps(sentences, indent = 4)))
#             # Creating the results list
#             if flag:
#                 documents_with_incentives += 1
#                 results_list[documents_with_incentives].append(name)
#                 results_list[documents_with_incentives].append(incentive_counter)
#                 for key, value in incentives_dict.items():
#                     results_list[documents_with_incentives].append(value)

# with open(f"../output/incentives_simple_transformers_{language}.csv", "w") as f:
#     writer = csv.writer(f)
#     writer.writerows(results_list)

ZeroDivisionError: division by zero

In [ ]:
j

In [ ]:
incentives_dict

In [ ]:
results_list

In [ ]:
sentences